# OpenCV Python for Face Recognition and Identification

Use python and opencv Haar Cascade face detector and LBPH face recognizer to build a face recognition system.


## Source:
https://www.youtube.com/watch?v=PmZ29Vta7Vc&list=PLIxHGHeOyd0zy51wDjY0wjVpAjo3E93ir&index=29

## 0. Import Libraries

In [1]:
import cv2
import os
import pickle
import numpy as np
from PIL import Image

## 1. Use openCV Haarcascade model for face detection

In [24]:
# Search your opencv location by running this print statement.
# My opencv location is : C:\Users\ACER\AppData\Roaming\Python\Python37\site-packages\cv2
# Go to a folder named "data" and copy the 'haarcascade_frontalface_alt2.xml' to your folder
print(cv2.__file__)

C:\Users\ACER\AppData\Roaming\Python\Python37\site-packages\cv2\cv2.cp37-win_amd64.pyd


In [36]:
# Import opencv buildin cascade classifier
face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_alt2.xml')

In [26]:
# Accessing web cam. Press q to quit the camera

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, minNeighbors = 5) #scaleFactor
    
    # x,y = start of the faces frame(top left)
    # w,h = width and height of frame
    for (x, y, w, h) in faces:

        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        #img_item = 'my_image.png'
        #cv2.imwrite(img_item, roi_gray)
        
        # Draw a rectangle around detected faces
        color = (255, 0, 0) #BGR (opencv default)
        stroke = 2 #line thickness
        end_cord_x = x+w
        end_cord_y = y+h
        cv2.rectangle(frame, (x,y), (end_cord_x, end_cord_y), color, stroke)

    # Display the resulting frame
    cv2.imshow('frame',frame)
    
    # Press q to quit/turn off camera
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [27]:
# Define image directory
image_dir = 'C:/Users/ACER/Desktop/learn_arief/opencv/face_image/'

# Initialize training set and labels
x_train = []
y_labels = []

# Create a dictionary to convert labels into numeric
current_id = 0
label_ids ={}

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('png') or file.endswith('jpg') or file.endswith('jpeg'):
            path = os.path.join(root, file)
            label = os.path.basename(root) #os.path.dirname(path)
            #path = path.replace('\\', '/')
            print(label, path)
            
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
            id_ = label_ids[label]
            print(label_ids)
            
            pil_image = Image.open(path).convert('L')
            ## Rezise image (if necessary)
            # size = (550, 550)
            # final_image = pil_image.resize(size, Image.ANTIALIAS)            
            image_array = np.array(pil_image, 'uint8') #change pil_image to final_image if you resize the image
            faces = face_cascade.detectMultiScale(gray, minNeighbors = 5)
                    
            # Detect faces in the image
            for (x,y,w,h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)

print(y_labels)                
print(x_train)                

arief_ramadhan C:/Users/ACER/Desktop/learn_arief/opencv/face_image/arief_ramadhan\arief_ramadhan_1.jpg
{'arief_ramadhan': 0}
arief_ramadhan C:/Users/ACER/Desktop/learn_arief/opencv/face_image/arief_ramadhan\arief_ramadhan_2.png
{'arief_ramadhan': 0}
arief_ramadhan C:/Users/ACER/Desktop/learn_arief/opencv/face_image/arief_ramadhan\arief_ramadhan_3.jpg
{'arief_ramadhan': 0}
arief_ramadhan C:/Users/ACER/Desktop/learn_arief/opencv/face_image/arief_ramadhan\arief_ramadhan_4.jpg
{'arief_ramadhan': 0}
arief_ramadhan C:/Users/ACER/Desktop/learn_arief/opencv/face_image/arief_ramadhan\arief_ramadhan_5.jpg
{'arief_ramadhan': 0}
bill_gates C:/Users/ACER/Desktop/learn_arief/opencv/face_image/bill_gates\bill_gates_1.jpg
{'arief_ramadhan': 0, 'bill_gates': 1}
bill_gates C:/Users/ACER/Desktop/learn_arief/opencv/face_image/bill_gates\bill_gates_2.jpg
{'arief_ramadhan': 0, 'bill_gates': 1}
bill_gates C:/Users/ACER/Desktop/learn_arief/opencv/face_image/bill_gates\bill_gates_3.jpg
{'arief_ramadhan': 0, 'b

In [28]:
# Save data label to a pickle file

with open('labels.pickle', 'wb') as f:
    pickle.dump(label_ids, f)

## 2. Use open CV face recognizer

In [29]:
# Import open cv recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()

In [30]:
# Train recognizer on the images
recognizer.train(x_train, np.array(y_labels))

# Save the trained recognizer
recognizer.save('trainer.yml')

## 3. Combine the face detector (Haar Cascade) and the Face Recognizer (LBPH) into one function 

In [37]:
# Import face detector and face recognizer
face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer.yml')

# Optional: use Haar Cascade for eyes. Get it from the same folder where you find the haarcascade_frontalface_alt2.xml
eye_cascade = cv2.CascadeClassifier('cascades/haarcascade_eye.xml')

In [31]:
# Inverse label
label = {v:k for k,v in label_ids.items()}
print(label[0], label[1], label[2])

arief_ramadhan bill_gates steve_wozniak


In [39]:
# Accessing web cam. Press q to quit the camera

## Initialize web cam 
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, minNeighbors = 5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        id_, conf = recognizer.predict(roi_gray)
        
        img_item = 'my_image.png'
        cv2.imwrite(img_item, roi_gray)
        
        if conf >= 45:
            print(id_)
            print(label[id_])
            
            # Draw labels on the Haar Cascade rectangle
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = label[id_]
            color = (255, 255, 255)
            stroke = 2
            cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)
        
        # Draw a rectangle around detected faces
        color = (255, 0, 0) #BGR (opencv default)
        stroke = 2 #line thickness
        end_cord_x = x+w
        end_cord_y = y+h
        cv2.rectangle(frame, (x,y), (end_cord_x, end_cord_y), color, stroke)
        
        ## Optional: create rectangles for eyes
#         eyes = eye_cascade.detectMultiScale(roi_gray)
#         for (ex, ey, ew, eh) in eyes:
#             cv2.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh), (0,255,0), 2)
            
    # Display the resulting frame
    cv2.imshow('frame',frame)
    
    # Press q to quit/turn off camera
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadhan
0
arief_ramadh